In [2]:
import numpy as np
import xarray as xr
import xesmf as xe
import glob
import os

In [43]:
def regrid_from_native(expname, varname):
    static = "/work/Katherine.Turner/tempest-extremes/odiv-251/data/tripolar_data/ocean_static.nc"
    fn = "/work/Katherine.Turner/tempest-extremes/odiv-251/data/tripolar_data/bilinear_540x720_180x360.nc"

    ds_static = xr.open_dataset(static,decode_times=False)
    
    files_to_regrid = sorted(glob.glob(f"/work/Katherine.Turner/tempest-extremes/{expname}/data/tripolar_data/*{varname}.nc"))
    
    for i in np.arange(1):#len(files_to_regrid)):
        # check to see if file exists in output
        file_out_tail = files_to_regrid[i].split("/")[-1][:-3]+".regrid.nc"
        file_out = f"/work/Katherine.Turner/tempest-extremes/{expname}/data/{file_out_tail}"  
        
        if os.path.isfile(file_out):
            print(f"{file_out_tail} exists. Continuing...")
        
        else:
            print(f"{file_out_tail} does not exist. Regridding...")
            ds = xr.open_dataset(files_to_regrid[i], use_cftime=True)
            
            if ('yh' in list(ds.dims)) & ('xh' in list(ds.dims)):
                    ds_grid = xr.Dataset({
                        "lon_b":xr.DataArray(ds_static["geolon"].values[::2,::2],dims=("yhb","xhb")),
                        "lat_b":xr.DataArray(ds_static["geolat"].values[::2,::2],dims=("yhb","xhb")),
                        "lon":xr.DataArray(ds_static["geolon"].values[1::2,1::2],dims=("yh","xh")),
                        "lat":xr.DataArray(ds_static["geolat"].values[1::2,1::2],dims=("yh","xh")),
                    })
            elif ('yh' in list(ds.dims)) & ('xq' in list(ds.dims)):
                    ds_grid = xr.Dataset({
                        "lon_b":xr.DataArray(ds_static["geolon_u"].values[::2,::2],dims=("yhb","xqb")),
                        "lat_b":xr.DataArray(ds_static["geolat_u"].values[::2,::2],dims=("yhb","xqb")),
                        "lon":xr.DataArray(ds_static["geolon_u"].values[1::2,1::2],dims=("yh","xq")),
                        "lat":xr.DataArray(ds_static["geolat_u"].values[1::2,1::2],dims=("yh","xq")),
                    })
            elif ('yq' in list(ds.dims)) & ('xh' in list(ds.dims)):
                    ds_grid = xr.Dataset({
                        "lon_b":xr.DataArray(ds_static["geolon_v"].values[::2,::2],dims=("yqb","xhb")),
                        "lat_b":xr.DataArray(ds_static["geolat_v"].values[::2,::2],dims=("yqb","xhb")),
                        "lon":xr.DataArray(ds_static["geolon_v"].values[1::2,1::2],dims=("yq","xh")),
                        "lat":xr.DataArray(ds_static["geolat_v"].values[1::2,1::2],dims=("yq","xh")),
                    })
            elif ('yq' in list(ds.dims)) & ('xq' in list(ds.dims)):
                    ds_grid = xr.Dataset({
                        "lon_b":xr.DataArray(ds_static["geolon_c"].values[::2,::2],dims=("yqb","xqb")),
                        "lat_b":xr.DataArray(ds_static["geolat_c"].values[::2,::2],dims=("yqb","xqb")),
                        "lon":xr.DataArray(ds_static["geolon_c"].values[1::2,1::2],dims=("yq","xq")),
                        "lat":xr.DataArray(ds_static["geolat_c"].values[1::2,1::2],dims=("yq","xq")),
                    })
            else:
                print("Unclear coordinates used for native grid, ignoring")
                return
                    
                    
            ds_interim = xr.Dataset({
                varname:ds[varname][:,1::2,1::2].assign_coords({"time":ds.time,"lat":ds_grid.lat,"lon":ds_grid.lon}),
                "lat_b":ds_grid.lat_b,
                "lon_b":ds_grid.lon_b,
            })
            
            ycoord = list(ds.dims)[-2]
            xcoord = list(ds.dims)[-1]
            
            ds_interim = ds_interim.assign_coords({xcoord:np.arange(1,len(ds_interim[xcoord])+1), ycoord:np.arange(1,len(ds_interim[ycoord])+1)})
            ds_atmos = xe.util.grid_global(1.25, 1)
            
            regridder = xe.Regridder(ds_interim, ds_atmos, "bilinear", weights=fn)
            dr_out = regridder(ds_interim[varname], keep_attrs=True)
            
            ds_out = xr.Dataset({varname:dr_out})
            ds_out.to_netcdf(fileout)    

In [55]:
expname='odiv-251'
varname='ssv'
    
static = "/work/Katherine.Turner/tempest-extremes/odiv-251/data/tripolar_data/ocean_static.nc"
fn = "/work/Katherine.Turner/tempest-extremes/odiv-251/data/tripolar_data/bilinear_540x720_180x360.nc"

ds_static = xr.open_dataset(static,decode_times=False)
    
files_to_regrid = sorted(glob.glob(f"/work/Katherine.Turner/tempest-extremes/{expname}/data/tripolar_data/*{varname}.nc"))
    
for i in np.arange(1):#len(files_to_regrid)):
    # check to see if file exists in output
    file_out_tail = files_to_regrid[i].split("/")[-1][:-3]+".regrid.nc"
    file_out = f"/work/Katherine.Turner/tempest-extremes/{expname}/data/{file_out_tail}"  
        
    if os.path.isfile(file_out):
        print(f"{file_out_tail} exists. Continuing...")
        
    else:
        print(f"{file_out_tail} does not exist. Regridding...")
        ds = xr.open_dataset(files_to_regrid[i], use_cftime=True)
            
        if ('yh' in list(ds.dims)) & ('xh' in list(ds.dims)):
                ds_grid = xr.Dataset({
                    "lon_b":xr.DataArray(ds_static["geolon"].values[::2,::2],dims=("yhb","xhb")),
                    "lat_b":xr.DataArray(ds_static["geolat"].values[::2,::2],dims=("yhb","xhb")),
                    "lon":xr.DataArray(ds_static["geolon"].values[1::2,1::2],dims=("yh","xh")),
                    "lat":xr.DataArray(ds_static["geolat"].values[1::2,1::2],dims=("yh","xh")),
                })
                ds_interim = xr.Dataset({
                    varname:ds[varname][:,1::2,1::2].assign_coords({"time":ds.time,"lat":ds_grid.lat,"lon":ds_grid.lon}),
                    "lat_b":ds_grid.lat_b,
                    "lon_b":ds_grid.lon_b,
                })
                ds_interim = ds2.assign_coords({"xh":np.arange(1,len(ds_interim.xh)+1), "yh":np.arange(1,len(ds_interim.yh)+1)})
                
        elif ('yh' in list(ds.dims)) & ('xq' in list(ds.dims)):
                ds_grid = xr.Dataset({
                    "lon_b":xr.DataArray(ds_static["geolon_u"].values[::2,::2],dims=("yhb","xqb")),
                    "lat_b":xr.DataArray(ds_static["geolat_u"].values[::2,::2],dims=("yhb","xqb")),
                    "lon":xr.DataArray(ds_static["geolon_u"].values[1::2,1::2],dims=("yh","xq")),
                    "lat":xr.DataArray(ds_static["geolat_u"].values[1::2,1::2],dims=("yh","xq")),
                })
                
        elif ('yq' in list(ds.dims)) & ('xh' in list(ds.dims)):
                ds_grid = xr.Dataset({
                    "lon_b":xr.DataArray(ds_static["geolon_v"].values[::2,::2],dims=("yqb","xhb")),
                    "lat_b":xr.DataArray(ds_static["geolat_v"].values[::2,::2],dims=("yqb","xhb")),
                    "lon":xr.DataArray(ds_static["geolon_v"].values[1::2,1::2],dims=("yq","xh")),
                    "lat":xr.DataArray(ds_static["geolat_v"].values[1::2,1::2],dims=("yq","xh")),
                })
                ds_interim = xr.Dataset({
                    varname:ds[varname][:,1::2,1::2].assign_coords({"time":ds.time,"lat":ds_grid.lat,"lon":ds_grid.lon}),
                    "lat_b":ds_grid.lat_b,
                    "lon_b":ds_grid.lon_b,
                })
                ds_interim = ds2.assign_coords({"xh":np.arange(1,len(ds_interim.xh)+1), "yq":np.arange(1,len(ds_interim.yq)+1)})
                
        elif ('yq' in list(ds.dims)) & ('xq' in list(ds.dims)):
                ds_grid = xr.Dataset({
                    "lon_b":xr.DataArray(ds_static["geolon_c"].values[::2,::2],dims=("yqb","xqb")),
                    "lat_b":xr.DataArray(ds_static["geolat_c"].values[::2,::2],dims=("yqb","xqb")),
                    "lon":xr.DataArray(ds_static["geolon_c"].values[1::2,1::2],dims=("yq","xq")),
                    "lat":xr.DataArray(ds_static["geolat_c"].values[1::2,1::2],dims=("yq","xq")),
                })
        else:
            print("Unclear coordinates used for native grid, ignoring")
          

ocean_daily_gfdl.01510101-01551231.ssv.regrid.nc does not exist. Regridding...


In [66]:
ds_interim = xr.Dataset({
        varname:ds[varname][:,1::2,1::2].assign_coords({"time":ds.time,"lat":ds_grid.lat,"lon":ds_grid.lon}),
        "lat_b":ds_grid.lat_b,
        "lon_b":ds_grid.lon_b,
    })
            
ycoord = list(ds.dims)[-2]
xcoord = list(ds.dims)[-1]
            
ds_interim = ds_interim.assign_coords({xcoord:np.arange(1,len(ds_interim[xcoord])+1), ycoord:np.arange(1,len(ds_interim[ycoord])+1)})
ds_atmos = xe.util.grid_global(1.25, 1, lon1=60)
            

In [72]:
ycoord

'yq'

In [73]:
regridder = xe.Regridder(ds_interim, ds_atmos, "bilinear", weights=fn)
dr_out = regridder(ds_interim[varname], keep_attrs=True)
         

ValueError: invalid entry in coordinates array

In [69]:
   
ds_out = xr.Dataset({varname:dr_out})
ds_out.to_netcdf(fileout)    

/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


ValueError: invalid entry in coordinates array

In [32]:
static = "/work/Katherine.Turner/tempest-extremes/odiv-251/data/tripolar_data/ocean_static.nc"
fn = "bilinear_540x720_180x360.nc"

ds_static = xr.open_dataset(static,decode_times=False)
ds_static

<xarray.Dataset>
Dimensions:       (xh: 1440, yh: 1080, time: 1, xq: 1440, yq: 1080)
Coordinates:
  * xh            (xh) float64 -299.7 -299.5 -299.2 -299.0 ... 59.53 59.78 60.03
  * yh            (yh) float64 -80.39 -80.31 -80.23 -80.15 ... 89.73 89.84 89.95
  * time          (time) float64 0.0
  * xq            (xq) float64 -299.6 -299.3 -299.1 -298.9 ... 59.66 59.91 60.16
  * yq            (yq) float64 -80.35 -80.27 -80.19 -80.11 ... 89.78 89.89 90.0
Data variables: (12/27)
    areacello     (yh, xh) float32 ...
    deptho        (yh, xh) float32 ...
    hfgeou        (yh, xh) float32 ...
    sftof         (yh, xh) float32 ...
    Coriolis      (yq, xq) float32 ...
    geolon        (yh, xh) float32 ...
    ...            ...
    dxCv          (yq, xh) float32 ...
    dyCv          (yq, xh) float32 ...
    areacello_cu  (yh, xq) float32 ...
    areacello_cv  (yq, xh) float32 ...
    areacello_bu  (yq, xq) float32 ...
    basin         (yh, xh) int32 ...
Attributes:
    filename:   19000101.ocean_static.nc
    title:      OM4_SIS2_cgrid_025
    grid_type:  regular
    grid_tile:  N/A

In [11]:
expname = "odiv-251"
varname = "ssu"
files_to_regrid = sorted(glob.glob(f"/work/Katherine.Turner/tempest-extremes/{expname}/data/tripolar_data/*{varname}.nc"))

i=0

file_out_tail = files_to_regrid[i].split("/")[-1][:-3]+".regrid.nc"
file_out = f"/work/Katherine.Turner/tempest-extremes/{expname}/data/{file_out_tail}"
file_out

ds = xr.open_dataset(files_to_regrid[i], use_cftime=True)

In [33]:
list(ds.dims)

['time', 'nv', 'yh', 'xq']

In [18]:
for i in start_year:
    ds = xr.open_dataset(f"ocean_daily_gfdl.0{i}0101-0{i+4}1231.{var}.nc" ,use_cftime=True)

    ds_grid = xr.Dataset({
        "lon_b":xr.DataArray(ds_static["geolon_u"].values[::2,::2],dims=("yqb","xhb")),
        "lat_b":xr.DataArray(ds_static["geolat_u"].values[::2,::2],dims=("yqb","xhb")),
        "lon":xr.DataArray(ds_static["geolon_u"].values[1::2,1::2],dims=("yq","xh")),
        "lat":xr.DataArray(ds_static["geolat_u"].values[1::2,1::2],dims=("yq","xh")),
    })

    ds2 = xr.Dataset({
        "ssv":ds.ssv[:,1::2,1::2].assign_coords({"time":ds.time,"lat":ds_grid.lat,"lon":ds_grid.lon}),
        "lat_b":ds_grid.lat_b,
        "lon_b":ds_grid.lon_b,
    })

    ds2 = ds2.assign_coords({"xh":np.arange(1,len(ds2.xh)+1), "yq":np.arange(1,len(ds2.yq)+1)})
    ds_out = xe.util.grid_global(1, 1)

    regridder = xe.Regridder(ds2, ds_out, "bilinear", weights=fn)
    dr_out = regridder(ds2[var], keep_attrs=True)

    dsout = xr.Dataset({"ssv":dr_out})
    dsout.to_netcdf(f"../ocean_daily_gfdl.1x1regrid.0{i}0101-0{i+4}1231.{var}.nc")
    print(i)

/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


151


/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


156


/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


...

231


/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


236


/nbhome/Katherine.Turner/miniconda3/envs/py310/lib/python3.10/site-packages/xesmf/backend.py:41: UserWarning: Input array is not F_CONTIGUOUS. Will affect performance.
  warnings.warn('Input array is not F_CONTIGUOUS. ' 'Will affect performance.')


241


In [ ]:
ds.tauvo[0,:,:].plot()

In [ ]:
dsout.tauvo[0,:,:].plot()